# dim_messenger

### Importación de librerías

In [1]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
messenger = pd.read_sql_table('clientes_mensajeroaquitoy', engineSource)
messenger_info = pd.read_sql_table('auth_user', engineSource)
messenger.drop(columns=["id", "fecha_entrada", "fecha_salida", "salario", "ciudad_operacion_id", "token_Firebase", "url_foto", "activo"], inplace=True)
messenger_info.drop(columns=["password", "last_login", "is_superuser", "username", "last_name", "is_staff", "is_active", "date_joined"], inplace=True)
messenger = messenger.merge(messenger_info, left_on="user_id", right_on="id", how="left")
messenger.drop(columns=["id"], inplace=True)
messenger.rename(columns={
  "first_name": "name",
  "telefono": "phone",
  "user_id": "key_dim_messenger"}, inplace=True)

messenger

### Carga de datos

In [ ]:
messenger.to_sql("dim_messenger", engineDestination, index=False, if_exists="replace")